In [2]:
import torch
from torch import nn

from torch.utils.tensorboard import SummaryWriter

import string
import random
import sys
import math
import unidecode

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [12]:
all_characters = string.printable
n_characters = len(all_characters)
all_characters

'0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c'